**Detecting tweets related to disasters**

In this example we will use lazytextpredict to determine the best model to detect whether tweets relate to disasters or not. The training data is provided by [Appen](https://appen.com/datasets/combined-disaster-response-data/), and uses english translations of multilingual tweets.

To get started we install lazytextpredict...

In [ ]:
!pip install lazy-text-predict

     |████████████████████████████████| 1.7MB 8.7MB/s 
     |████████████████████████████████| 1.2MB 38.9MB/s 
     |████████████████████████████████| 20.1MB 1.4MB/s 
     |████████████████████████████████| 1.3MB 59.8MB/s 
     |████████████████████████████████| 6.8MB 60.6MB/s 
     |████████████████████████████████| 245kB 53.5MB/s 
     |████████████████████████████████| 17.7MB 136kB/s 
     |████████████████████████████████| 890kB 58.4MB/s 
     |████████████████████████████████| 2.9MB 67.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=f956f91d68fe1df37e228b067617e13264fcf8d294a4307429de61db63937fc2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: tensorflow 2.4.0 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which 

**Download data**

Next we have to download the data from the appen website:

In [ ]:
!wget https://datasets.appen.com/appen_datasets/disaster_response_data/disaster_response_messages_training.csv
!wget https://datasets.appen.com/appen_datasets/disaster_response_data/disaster_response_messages_test.csv
!wget https://datasets.appen.com/appen_datasets/disaster_response_data/disaster_response_messages_validation.csv

--2021-01-16 14:22:46--  https://datasets.appen.com/appen_datasets/disaster_response_data/disaster_response_messages_training.csv
Resolving datasets.appen.com (datasets.appen.com)... 54.210.240.39, 54.210.154.205, 54.86.241.79
Connecting to datasets.appen.com (datasets.appen.com)|54.210.240.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5746561 (5.5M) [text/csv]
Saving to: ‘disaster_response_messages_training.csv’

disaster_response_m 100%[===================>]   5.48M  10.6MB/s    in 0.5s    

2021-01-16 14:22:47 (10.6 MB/s) - ‘disaster_response_messages_training.csv’ saved [5746561/5746561]

--2021-01-16 14:22:47--  https://datasets.appen.com/appen_datasets/disaster_response_data/disaster_response_messages_test.csv
Resolving datasets.appen.com (datasets.appen.com)... 54.210.240.39, 54.210.154.205, 54.86.241.79
Connecting to datasets.appen.com (datasets.appen.com)|54.210.240.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7154

**Data handling**

Now that we have the data we need to read the downloaded data into pandas dataframes. These dataframes are concatenated into one big dataset, which will be split into train and test sets by lazytextpredict.

In [ ]:
import pandas as pd
training_data=pd.read_csv(filepath_or_buffer='/content/disaster_response_messages_training.csv')
test_data=pd.read_csv(filepath_or_buffer='/content/disaster_response_messages_test.csv')
validation_data=pd.read_csv(filepath_or_buffer='/content/disaster_response_messages_validation.csv')
data=training_data.append(test_data)
data=data.append(validation_data) ## put all the data into one df so that test_train_split can stratify it properly 
##(I included the verification data because there is no method included in lazytextpredict at the moment)
data = data[data.related != 2]
data.head()


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,split,message,original,genre,related,PII,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,train,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,train,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,12,train,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,14,train,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,15,train,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0


**Initial training**

Now that we have the data we can put it into lazytextpredict. 

We do this by creating an instance of the LTP class with 

```Xdata=data['message'] ``` i.e. the translated tweet text

and 

``` Ydata=data['related'] ``` i.e. if the message is related to a disaster.

We train all the models with a small number of training epochs (5) to get an idea of how effective our models are with this data. This should take about 20 minutes.


In [ ]:
from lazytextpredict import basic_classification
trial=basic_classification.LTP(Xdata=data['message'],Ydata=data['related'])
trial.run(training_epochs=5)

converting pandas series to list
converting pandas series to list
X_train length: 1302
X_test length: 1303
Y_train length: 1302
Y_test length: 1303
Training on a dataset with 2 labels


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

/usr/local/lib/python3.6/dist-packages/nlp/utils/py_utils.py:191: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return function(data_struct)


Step,Training Loss


Trainer is attempting to log a value of "{'0': {'precision': 0.7843137254901961, 'recall': 0.39215686274509803, 'f1-score': 0.522875816993464, 'support': 306}, '1': {'precision': 0.8382608695652174, 'recall': 0.966900702106319, 'f1-score': 0.8979972054028877, 'support': 997}, 'accuracy': 0.8319263238679969, 'macro avg': {'precision': 0.8112872975277068, 'recall': 0.6795287824257085, 'f1-score': 0.7104365111981759, 'support': 1303}, 'weighted avg': {'precision': 0.8255917781707763, 'recall': 0.8319263238679969, 'f1-score': 0.8099026966896999, 'support': 1303}}" of type <class 'dict'> for key "eval/full_report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


...

{'eval_loss': 0.4899747669696808, 'eval_accuracy': 0.7620874904067536, 'eval_f1': 0.7554408102865675, 'eval_precision': 0.7506871758625725, 'eval_recall': 0.7620874904067536, 'eval_full_report': {'0': {'precision': 0.49236641221374045, 'recall': 0.4215686274509804, 'f1-score': 0.4542253521126761, 'support': 306}, '1': {'precision': 0.829971181556196, 'recall': 0.8665997993981945, 'f1-score': 0.8478900883218843, 'support': 997}, 'accuracy': 0.7620874904067536, 'macro avg': {'precision': 0.6611687968849682, 'recall': 0.6440842134245874, 'f1-score': 0.6510577202172803, 'support': 1303}, 'weighted avg': {'precision': 0.7506871758625725, 'recall': 0.7620874904067536, 'f1-score': 0.7554408102865675, 'support': 1303}}, 'epoch': 5.0}
Training on a dataset with 2 labels


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Step,Training Loss


Trainer is attempting to log a value of "{'0': {'precision': 0.7412935323383084, 'recall': 0.4869281045751634, 'f1-score': 0.5877712031558185, 'support': 306}, '1': {'precision': 0.8575317604355717, 'recall': 0.9478435305917753, 'f1-score': 0.9004287756074322, 'support': 997}, 'accuracy': 0.8396009209516501, 'macro avg': {'precision': 0.79941264638694, 'recall': 0.7173858175834693, 'f1-score': 0.7440999893816254, 'support': 1303}, 'weighted avg': {'precision': 0.8302340645048254, 'recall': 0.8396009209516501, 'f1-score': 0.8270034362596244, 'support': 1303}}" of type <class 'dict'> for key "eval/full_report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.574372410774231, 'eval_accuracy': 0.8396009209516501, 'eval_f1': 0.8270034362596244, 'eval_precision': 0.8302340645048254, 'eval_recall': 0.8396009209516501, 'eval_full_report': {'0': {'precision': 0.7412935323383084, 'recall': 0.4869281045751634, 'f1-score': 0.5877712031558185, 'support': 306}, '1': {'precision': 0.8575317604355717, 'recall': 0.9478435305917753, 'f1-score': 0.9004287756074322, 'support': 997}, 'accuracy': 0.8396009209516501, 'macro avg': {'precision': 0.79941264638694, 'recall': 0.7173858175834693, 'f1-score': 0.7440999893816254, 'support': 1303}, 'weighted avg': {'precision': 0.8302340645048254, 'recall': 0.8396009209516501, 'f1-score': 0.8270034362596244, 'support': 1303}}, 'epoch': 5.0}
Training on a dataset with 2 labels
ERROR
best parameters are:
{'clf__alpha': 0.001, 'clf__penalty': 'l2', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
{'eval_loss': 0.21105141980046047, 'eval_accuracy': 0.7889485801995395, 'eval_f1': 0.5678588011119352, 'eva

**Analyze results**

Let's compare the models using the print_metrics_table method, and a couple of dummy tweets. We can see that roberta-base performs a little better than all the other models on this challenge, so we'll use that going forward with focused training.

In [ ]:
trial.print_metrics_table()

                    Model            loss        accuracy              f1       precision          recall
        bert-base-uncased          0.8547         0.83193          0.8099         0.82559         0.83193
           albert-base-v2         0.48997         0.76209         0.75544         0.75069         0.76209
             roberta-base         0.57437          0.8396           0.827         0.83023          0.8396
               linear_SVM         0.21105         0.78895         0.56786         0.76503         0.56991
multinomial_naive_bayesian         0.21642         0.78358         0.54556         0.75423          0.5562


In [ ]:
trial.predict(text="Cardi B's new album is the best")

bert-base-uncased
{'label': 'LABEL_0', 'score': 0.9794496297836304}
albert-base-v2
{'label': 'LABEL_1', 'score': 0.5572738647460938}
roberta-base
{'label': 'LABEL_0', 'score': 0.690045952796936}
linear_SVM
[1]
multinomial_naive_bayesian
[1]


In [ ]:
trial.predict(text="All good now, just needed some R&R")

bert-base-uncased
{'label': 'LABEL_0', 'score': 0.9884157776832581}
albert-base-v2
{'label': 'LABEL_0', 'score': 0.8769664168357849}
roberta-base
{'label': 'LABEL_1', 'score': 0.9080235362052917}
linear_SVM
[1]
multinomial_naive_bayesian
[1]


In [ ]:
trial.predict(text="this rally is getting out of hand")

bert-base-uncased
{'label': 'LABEL_1', 'score': 0.9860490560531616}
albert-base-v2
{'label': 'LABEL_0', 'score': 0.7396048307418823}
roberta-base
{'label': 'LABEL_1', 'score': 0.8055053353309631}
linear_SVM
[1]
multinomial_naive_bayesian
[1]


In [ ]:
trial.predict(text="SOS, flood warning in Gatineau. Too much gravy on our poutine!")

bert-base-uncased
{'label': 'LABEL_1', 'score': 0.9996545314788818}
albert-base-v2
{'label': 'LABEL_1', 'score': 0.9297879934310913}
roberta-base
{'label': 'LABEL_1', 'score': 0.9968279600143433}
linear_SVM
[1]
multinomial_naive_bayesian
[1]


**Focused training**

Now that we have identified the model we want to use we can train and test it specifically...

In [ ]:
focused_trial=basic_classification.LTP(Xdata=data['message'],Ydata=data['related'],test_frac=0.1,train_frac=0.9)
focused_trial.run(focused=True,focused_model='roberta-base',training_epochs=10)

TypeError: ignored

In [ ]:
focused_trial.print_metrics_table()

                    Model            loss        accuracy              f1       precision          recall
             roberta-base         0.61509         0.84539         0.84042         0.83897         0.84539


In [ ]:
trial.predict(model_name='roberta-base', focused=True,text='This movie was really something. I loved every second of it')

roberta-base
{'label': 'LABEL_0', 'score': 0.990229070186615}


In [ ]:
trial.predict(model_name='roberta-base', focused=True,text='Oh no, trouble is brewing')

roberta-base
{'label': 'LABEL_1', 'score': 0.9322614073753357}


In [ ]:
trial.predict(model_name='roberta-base', focused=True,text='Get out of town! A maple syrup disaster!')

roberta-base
{'label': 'LABEL_1', 'score': 0.9910778403282166}


In [ ]:
trial.predict(model_name='roberta-base', focused=True,text='Watch out, jack frost is about')

roberta-base
{'label': 'LABEL_1', 'score': 0.9950404763221741}
